# Importando Pacotes

In [1]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
import strsimpy
import numpy as np
from strsimpy.normalized_levenshtein import NormalizedLevenshtein
from strsimpy.jaro_winkler import JaroWinkler
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors

# Paths

In [2]:
path_bases = "C:\\Users\\aamma\\OneDrive\\ENCE\\TCC\\REFAZENDO EXPERIMENTO\\Bases"
path_embeddings = "C:\\Users\\aamma\\OneDrive\\ENCE\\TCC\\REFAZENDO EXPERIMENTO\\Embeddings"
path_resultados = "C:\\Users\\aamma\\OneDrive\\ENCE\\TCC\\REFAZENDO EXPERIMENTO\\Resultados"

# Leitura de Bases

In [3]:
os.chdir(path_bases)
descricoes_pof = pd.read_excel("Descricoes_POF_g4.xlsx")
descricoes_snipc = pd.read_excel("Descricoes_SNIPC_g4.xlsx")
descricoes_pof = descricoes_pof.values.tolist() # Converte dataframe para lista de listas
descricoes_snipc = descricoes_snipc.values.tolist() # Coverte dataframe para lista de listas
descricoes_pof = [item for sublist in descricoes_pof for item in sublist] # Converte lista de listas para lista
descricoes_snipc = [item for sublist in descricoes_snipc for item in sublist] # Converte lista de listas para lista

# Criando Matrizes de Similaridade

In [4]:
## Levenshtein
df_levenshtein = np.zeros((len(descricoes_pof), len(descricoes_snipc)))
df_levenshtein = pd.DataFrame(df_levenshtein, columns = descricoes_snipc,
index = descricoes_pof)

## Jaro
df_jaro = np.zeros((len(descricoes_pof), len(descricoes_snipc)))
df_jaro = pd.DataFrame(df_jaro, columns = descricoes_snipc,
index = descricoes_pof)

## Jaccard
df_jaccard = np.zeros((len(descricoes_pof), len(descricoes_snipc)))
df_jaccard = pd.DataFrame(df_jaccard, columns = descricoes_snipc,
index = descricoes_pof)

## TF-IDF
df_tfidf = np.zeros((len(descricoes_pof), len(descricoes_snipc)))
df_tfidf = pd.DataFrame(df_tfidf, columns = descricoes_snipc,
index = descricoes_pof)

## Word2Vec
df_word2vec_soma = np.zeros((len(descricoes_pof), len(descricoes_snipc)))
df_word2vec_soma = pd.DataFrame(df_word2vec_soma, columns = descricoes_snipc,
index = descricoes_pof)
df_word2vec_media = np.zeros((len(descricoes_pof), len(descricoes_snipc)))
df_word2vec_media = pd.DataFrame(df_word2vec_media, columns = descricoes_snipc,
index = descricoes_pof)

# Calculando Similaridade

## Levenshtein

In [5]:
levenshtein = NormalizedLevenshtein()
contador = 0
for palavra_pof in df_levenshtein.index:
    for palavra_snipc in df_levenshtein.columns:
        df_levenshtein.at[palavra_pof, palavra_snipc] = levenshtein.similarity(palavra_pof, palavra_snipc)
    contador = contador + 1
    if contador % 700 == 0 or contador == 3305:
        print(contador / df_levenshtein.shape[0] * 100, "%")

In [6]:
path_resultados = "C:\\Users\\aamma\\OneDrive\\ENCE\\TCC\\REFAZENDO EXPERIMENTO\\Resultados"
os.chdir(path_resultados)
df_levenshtein.to_excel("Levenshtein_g4_MS.xlsx")

## Jaro

In [7]:
jaro = JaroWinkler()
contador = 0
for palavra_pof in df_jaro.index:
    for palavra_snipc in df_jaro.columns:
        df_jaro.at[palavra_pof, palavra_snipc] = jaro.similarity(palavra_pof, palavra_snipc)
    contador = contador + 1
    if contador % 700 == 0 or contador == 3305:
        print(contador / df_jaro.shape[0] * 100, "%")

In [8]:
os.chdir(path_resultados)
df_jaro.to_excel("Jaro_g4_MS.xlsx")

## Jaccard

In [9]:
contador = 0
for palavra_pof in df_jaccard.index:
    for palavra_snipc in df_jaccard.columns:
        tok_pof = set(palavra_pof.split())
        tok_snipc = set(palavra_snipc.split())
        numerador = tok_pof & tok_snipc
        denominador = tok_pof | tok_snipc
        df_jaccard.at[palavra_pof, palavra_snipc] = len(numerador) / len(denominador)
    contador = contador + 1
    if contador % 700 == 0 or contador == 3305:
        print(contador / df_jaccard.shape[0] * 100, "%")

In [10]:
os.chdir(path_resultados)
df_jaccard.to_excel("Jaccard_g4_MS.xlsx")

## TF-IDF

In [11]:
vectorizer = TfidfVectorizer()

# Juntando Todas as Descrições e Removendo duplicatas
todas_descricoes = descricoes_pof + descricoes_snipc
todas_descricoes = pd.Series(todas_descricoes).drop_duplicates().to_list()

# Calculando valores TF-IDF
tfidf_matrix = vectorizer.fit_transform(todas_descricoes)

# Transformando Matriz TF-IDF em DF
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=todas_descricoes, columns=vectorizer.get_feature_names_out())

# Filtrando por pesquisas
tfidf_snipc = tfidf_df.filter(items = descricoes_snipc, axis = 0)
tfidf_pof = tfidf_df.filter(items = descricoes_pof, axis = 0)

## Calculando Cosseno e transformando em DF
cosine_sim = cosine_similarity(tfidf_pof, tfidf_snipc)
df_tfidf = pd.DataFrame(cosine_sim, index = descricoes_pof, columns = descricoes_snipc)

In [12]:
os.chdir(path_resultados)
df_tfidf.to_excel("TFIDF_g4_MS.xlsx")

# Word2vec

In [13]:
os.chdir(path_embeddings)
os.getcwd()
model = KeyedVectors.load_word2vec_format("skip_s300.txt")

In [14]:
os.chdir(path_bases)
tokens_all = pd.read_excel("Tokens_All_g4.xlsx")
tokens_all = tokens_all.values.tolist()
tokens_all = [item for sublist in tokens_all for item in sublist]
vocab_set = set(model.index_to_key)
keep_set = set(tokens_all)
drop_set = vocab_set - keep_set
for word in drop_set:
    del model.key_to_index[word]

In [15]:
tokens_nc = pd.read_excel("Tokens_All_NC_g4.xlsx")
tokens_nc = tokens_nc.values.tolist()
tokens_nc = [item for sublist in tokens_nc for item in sublist]

In [16]:
tokens_nc

['dominni', 'passadeiro', '15', 'tosadeira', 'totolec', 'trimania']

In [17]:
matriz_aux = np.zeros((300, ), dtype = "float32")
for token in tokens_nc:
    model[token] = matriz_aux

## Soma de Embeddings

In [18]:
def embedding_soma(sentence, model, num_features, word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype = "float32")
    for word in words:
        if word in word_set:
            feature_vec = np.add(feature_vec, model[word])
    model[sentence] = feature_vec

In [19]:
word_set = set(model.index_to_key)
todas_descricoes = descricoes_pof + descricoes_snipc
contador = 0
for descricao in todas_descricoes:
    embedding_soma(descricao, model, 300, word_set)
    if contador % 10 == 0 or contador == 4592:
        print(contador / len(todas_descricoes) * 100, "%")
    contador = contador + 1

0.0 %
2.525252525252525 %
5.05050505050505 %
7.575757575757576 %
10.1010101010101 %
12.626262626262626 %
15.151515151515152 %
17.67676767676768 %
20.2020202020202 %
22.727272727272727 %
25.252525252525253 %
27.77777777777778 %
30.303030303030305 %
32.82828282828283 %
35.35353535353536 %
37.878787878787875 %
40.4040404040404 %
42.92929292929293 %
45.45454545454545 %
47.97979797979798 %
50.505050505050505 %
53.03030303030303 %
55.55555555555556 %
58.080808080808076 %
60.60606060606061 %
63.13131313131313 %
65.65656565656566 %
68.18181818181817 %
70.70707070707071 %
73.23232323232324 %
75.75757575757575 %
78.28282828282829 %
80.8080808080808 %
83.33333333333334 %
85.85858585858585 %
88.38383838383838 %
90.9090909090909 %
93.43434343434343 %
95.95959595959596 %
98.48484848484848 %


In [20]:
contador = 0
for palavra_pof in df_word2vec_soma.index:
    for palavra_snipc in df_word2vec_soma.columns:
        df_word2vec_soma.at[palavra_pof, palavra_snipc] = model.similarity(palavra_pof, palavra_snipc)
    contador = contador + 1
    if contador % 10 == 0 or contador == 3376:
        print(contador / df_word2vec_soma.shape[0] * 100, "%")

3.571428571428571 %
7.142857142857142 %
10.714285714285714 %
14.285714285714285 %
17.857142857142858 %
21.428571428571427 %
25.0 %
28.57142857142857 %
32.142857142857146 %
35.714285714285715 %
39.285714285714285 %
42.857142857142854 %
46.42857142857143 %
50.0 %
53.57142857142857 %
57.14285714285714 %
60.71428571428571 %
64.28571428571429 %
67.85714285714286 %
71.42857142857143 %
75.0 %
78.57142857142857 %
82.14285714285714 %
85.71428571428571 %
89.28571428571429 %
92.85714285714286 %
96.42857142857143 %
100.0 %


In [21]:
os.chdir(path_resultados)
df_word2vec_soma.to_excel("Word2VecSoma_g4_MS.xlsx")

## Word2Vec Média

In [22]:
def embedding_media(sentence, model, num_features, word_set):
    words = sentence.split()
    n_words = 0
    feature_vec = np.zeros((num_features, ), dtype = "float32")
    for word in words:
        if word in word_set:
            feature_vec = np.add(feature_vec, model[word])
            n_words = n_words + 1
    model[sentence] = feature_vec / n_words

In [23]:
word_set = set(model.index_to_key)
todas_descricoes = descricoes_pof + descricoes_snipc
contador = 0
for descricao in todas_descricoes:
    embedding_media(descricao, model, 300, word_set)
    if contador % 10 == 0 or contador == 4592:
        print(contador / len(todas_descricoes) * 100, "%")
    contador = contador + 1

0.0 %
2.525252525252525 %
5.05050505050505 %
7.575757575757576 %
10.1010101010101 %
12.626262626262626 %
15.151515151515152 %
17.67676767676768 %
20.2020202020202 %
22.727272727272727 %
25.252525252525253 %
27.77777777777778 %
30.303030303030305 %
32.82828282828283 %
35.35353535353536 %
37.878787878787875 %
40.4040404040404 %
42.92929292929293 %
45.45454545454545 %
47.97979797979798 %
50.505050505050505 %
53.03030303030303 %
55.55555555555556 %
58.080808080808076 %
60.60606060606061 %
63.13131313131313 %
65.65656565656566 %
68.18181818181817 %
70.70707070707071 %
73.23232323232324 %
75.75757575757575 %
78.28282828282829 %
80.8080808080808 %
83.33333333333334 %
85.85858585858585 %
88.38383838383838 %
90.9090909090909 %
93.43434343434343 %
95.95959595959596 %
98.48484848484848 %


In [24]:
contador = 0
for palavra_pof in df_word2vec_media.index:
    for palavra_snipc in df_word2vec_media.columns:
        df_word2vec_media.at[palavra_pof, palavra_snipc] = model.similarity(palavra_pof, palavra_snipc)
    contador = contador + 1
    if contador % 10 == 0 or contador == 3376:
        print(contador / df_word2vec_media.shape[0] * 100, "%")

3.571428571428571 %
7.142857142857142 %
10.714285714285714 %
14.285714285714285 %
17.857142857142858 %
21.428571428571427 %
25.0 %
28.57142857142857 %
32.142857142857146 %
35.714285714285715 %
39.285714285714285 %
42.857142857142854 %
46.42857142857143 %
50.0 %
53.57142857142857 %
57.14285714285714 %
60.71428571428571 %
64.28571428571429 %
67.85714285714286 %
71.42857142857143 %
75.0 %
78.57142857142857 %
82.14285714285714 %
85.71428571428571 %
89.28571428571429 %
92.85714285714286 %
96.42857142857143 %
100.0 %


In [25]:
os.chdir(path_resultados)
df_word2vec_media.to_excel("Word2VecMedia_g4_MS.xlsx")

In [26]:
contador

280